#Weather Predictions using Humidity and Temperature Data
## Using Tensorflow, pandas, sklearn

Load Data

In [114]:
import pandas as pd

url = "https://raw.githubusercontent.com/Thanu10ekoon/Data/refs/heads/main/Colombo_Weather_Extended_3Months.csv"

w_data = pd.read_csv(url,encoding='latin1')

w_data.head()

,Date,Average Humidity (%),Average Temperature (Â°C),Wind Gusts (km/h),Cloud Cover (%),Pressure (hPa),Dew Point (Â°C),Rainfall
0,12/15/2024,67,27.2,10.9,80,1013.6,23.1,0
1,12/16/2024,83,32.0,20.8,66,1007.1,28.1,0
2,12/17/2024,70,27.9,22.3,58,1006.3,23.6,0
3,12/18/2024,70,27.7,22.1,61,1007.2,23.6,1
4,12/19/2024,71,27.8,12.5,26,1007.6,23.8,1


Split data set into x,y

In [115]:
from sklearn.preprocessing import StandardScaler

X = w_data.drop(columns=['Date', 'Rainfall'], axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = w_data['Rainfall']
print(y)

0     0
1     0
2     0
3     1
4     1
     ..
85    0
86    0
87    0
88    0
89    0
Name: Rainfall, Length: 90, dtype: int64


Split training and testing data

In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

Installing necessary dependancies

In [117]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout

Build Model

In [118]:
from keras.layers import BatchNormalization, LeakyReLU

model = Sequential()
model.add(Dense(units=64, input_dim=len(X_train[0]), kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(units=32, kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(units=16, kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(units=1, activation='sigmoid'))

model.summary()

c:\Users\thant\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_48 (Dense)                │ (None, 64)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_9 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_10 (LeakyReLU)      │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_11 (LeakyReLU)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,521 (13.75 KB)

 Trainable params: 3,297 (12.88 KB)

 Non-trainable params: 224 (896.00 B)

Precision Increase

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

optimizer = Adam(learning_rate=0.01)

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)


Compile Model

In [120]:
model.compile(loss='binary_crossentropy',optimizer='sgd',metrics=['accuracy'])

Train develeoped Model

In [121]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=200, batch_size=20)

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.5775 - loss: 1.5012 - val_accuracy: 0.5278 - val_loss: 1.4724
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6368 - loss: 1.4772 - val_accuracy: 0.5278 - val_loss: 1.4705
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6583 - loss: 1.4597 - val_accuracy: 0.5556 - val_loss: 1.4681
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6551 - loss: 1.3911 - val_accuracy: 0.5556 - val_loss: 1.4660
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7301 - loss: 1.3732 - val_accuracy: 0.5278 - val_loss: 1.4656
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7146 - loss: 1.3841 - val_accuracy: 0.5278 - val_loss: 1.4645
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6799 - loss: 1.3952 - val_accuracy: 0.5000 - val_loss: 1.4629
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7081 - loss: 1.3561 - val_accuracy: 0.5000 - val_loss:

Save model

In [122]:
model.save("rain_prediction_model2.h5")


Load trained model

In [111]:
model = load_model("rain_prediction_model.h5")


User App

In [58]:
import numpy as np
from keras.models import load_model

# Load the trained model
model = load_model("rain_prediction_model.h5")

# Prompt user for input
humidity = float(input("Enter the average humidity: "))
temperature = float(input("Enter the average temperature: "))

user_input = np.array([[humidity, temperature]])

prediction = model.predict(user_input)[0][0]

if prediction > 0.5:
    print(f"There is a high possibility of rain. (Probability: {prediction:.2f})")
else:
    print(f"There is a low possibility of rain. (Probability: {prediction:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
There is a high possibility of rain. (Probability: 0.70)
